This simple note bla bla bla bla

Getting the data

In [3]:
import urllib.request
urllib.request.urlretrieve('https://cernbox.cern.ch/remote.php/dav/public-files/FGVtNuNgiSO4VyZ/data.h5', 'data.h5')

('data.h5', <http.client.HTTPMessage at 0x7b910856e620>)

Reading the data

In [1]:
import pandas as pd
dataset= pd.read_hdf('./data.h5')

Dictionary to make the plots

In [2]:
#Data structure for selection and plotting
binInfo = dict()
binInfo['threshold1'] = 0.0
binInfo['threshold2'] = 0.0
binInfo['xynbinx'] = 160
binInfo['xynbiny'] = 160
binInfo['xznbinx'] = 160
binInfo['xznbinz'] = 160
binInfo['yznbiny'] = 160
binInfo['yznbinz'] = 160
binInfo['limitX'] = [-30, 30]
binInfo['limitY'] = [-30, 30]
binInfo['limitZ'] = [-5, 5]


This function returns the point of closest approach

In [3]:
import numpy as np

def getPoint(r1, r2, v1, v2):

    #Calculation of the closest point of approach
    cross_st = np.cross(v1, v2)
    cross_stnorm = np.linalg.norm(cross_st)
    vts = np.dot(v1, v2)
    if cross_stnorm < 1.0e-6 or vts < 1.0e-6:
        return False, [0, 0, 0]
    cross_sst = np.cross(v1, cross_st)
    DeltaR = r1 - r2
    xpoca2 = r2 - v2 * np.dot(DeltaR, cross_sst)/cross_stnorm**2
    xpoca1 = r1 + v1 * np.dot((xpoca2-r1), v1)/vts
    v = 0.5 * (xpoca1 + xpoca2)
    return True, v

Now looping the dataset

In [ ]:
ax = []
ay = []
az = []

# loop through the rows using iterrows()
for index, row in dataset.iterrows():
    #if index > 100:
    #    break
    r1 = np.asarray([row['x1'], row['y1'], row['z1']])
    r2 = np.asarray([row['x2'], row['y2'], row['z2']])
    v1 = np.asarray([row['vx1'], row['vy1'], row['vz1']])
    v2 = np.asarray([row['vx2'], row['vy2'], row['vz2']])
    dtx = row['dthetax']
    dty = row['dthetay']

    valid = False
    ###Apply here a simple angular selection
    if abs(dtx) > binInfo['threshold1'] or abs(dty) > binInfo['threshold2']:
        valid, v = getPoint(r1, r2, v1, v2)
        if not valid:
            continue
        if v[0] < binInfo['limitX'][0] or v[0] > binInfo['limitX'][1]:
            continue
        if v[1] < binInfo['limitY'][0] or v[1] > binInfo['limitY'][1]:
            continue
        if v[2] < binInfo['limitZ'][0] or v[2] > binInfo['limitZ'][1]:
            continue
        ax.append(v[0])
        ay.append(v[1])
        az.append(v[2])
    else:
        continue
    x = np.asarray(ax)
    y = np.asarray(ay)
    z = np.asarray(az)

Create all the canvases and plot

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 3, figsize=(16, 5))
ax[0].set_title('Frontal view XY')
ax[1].set_title('Side view XZ')
ax[2].set_title('Side view YZ')
ax[0].set_xlabel('X [cm]')
ax[0].set_ylabel('Y [cm]')
ax[1].set_xlabel('X [cm]')
ax[1].set_ylabel('Z [cm]')
ax[2].set_xlabel('Y [cm]')
ax[2].set_ylabel('Z [cm]')

ax[0].hist2d(x, y, bins=(binInfo['xynbinx'], binInfo['xynbiny']), cmap=plt.cm.jet)
ax[1].hist2d(x, z, bins=(binInfo['xznbinx'], binInfo['xznbinz']), cmap=plt.cm.jet)
ax[2].hist2d(y, z, bins=(binInfo['yznbiny'], binInfo['yznbinz']), cmap=plt.cm.jet)
